In [243]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.models import Model
from keras.models import model_from_json

from keras.layers import Dense,LSTM, SpatialDropout1D,Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, Input 
from keras.layers.normalization import BatchNormalization

from sklearn.ensemble import ExtraTreesClassifier

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import utils

from nltk.tokenize import word_tokenize



%matplotlib inline

In [244]:
def extract(coinname):
    df = pd.read_csv('{}.csv'.format(coinname))
    df = df.replace(np.inf, np.nan)
    
    return df

In [245]:
BTC = extract('BTCUSD')

In [246]:
BTC['Dec'] = BTC['Topic'] + BTC['Text']

In [247]:
ETH =extract('ETHUSD')

In [248]:
ETH['Dec'] = ETH['Topic'] + ETH['Text']

In [249]:
LTC =extract('LTCUSD')

In [250]:
LTC['Dec'] = LTC['Topic'] + LTC['Text']

In [251]:
XRP =extract('XRPUSD')

In [252]:
XRP['Dec'] = XRP['Topic'] + XRP['Text']

In [253]:
DASH =extract('DASHUSD')

In [254]:
DASH['Dec'] = DASH['Topic'] + DASH['Text']

In [255]:
DOGE =extract('DOGEUSD')

In [256]:
DOGE['Dec'] = DOGE['Topic'] + DOGE['Text']

In [257]:
STR =extract('STRUSD')

In [258]:
STR['Dec'] = STR['Topic'] + STR['Text']

In [259]:
XMR =extract('XMRUSD')

In [260]:
XMR['Dec'] = XMR['Topic'] + XMR['Text']

In [261]:
coins = {}

coins['BTC'] = BTC
coins['LTC'] = LTC
coins['XMR'] = XMR
coins['DOGE'] = DOGE
coins['STR'] = STR
coins['DASH'] = DASH
coins['ETH'] = ETH
coins['XRP'] = XRP

In [262]:
df = pd.DataFrame(columns=['Time Stamp',  'Dec',  'Direction'])

b = BTC[(BTC['Direction'] != 'Long') & (BTC['Direction'] != 'Short')][['Dec', 'Direction', 'Unnamed: 0']]
b['coinname'] = 'BTC'

e = ETH[(ETH['Direction'] != 'Long') & (ETH['Direction'] != 'Short')][['Dec', 'Direction', 'Unnamed: 0']]
e['coinname'] = 'ETH'

l = LTC[(LTC['Direction'] != 'Long') & (LTC['Direction'] != 'Short')][['Dec', 'Direction', 'Unnamed: 0']]
l['coinname'] = 'LTC'

d = DASH[(DASH['Direction'] != 'Long') & (DASH['Direction'] != 'Short')][['Dec', 'Direction', 'Unnamed: 0']]
d['coinname'] = 'DASH'

x = XRP[(XRP['Direction'] != 'Long') & (XRP['Direction'] != 'Short')][['Dec', 'Direction', 'Unnamed: 0']]
x['coinname'] = 'XRP'

do = DOGE[(DOGE['Direction'] != 'Long') & (DOGE['Direction'] != 'Short')][['Dec', 'Direction', 'Unnamed: 0']]
do['coinname'] = 'DOGE'

s = STR[(STR['Direction'] != 'Long') & (STR['Direction'] != 'Short')][['Dec', 'Direction', 'Unnamed: 0']]
s['coinname'] = 'STR'

xm = XMR[(XMR['Direction'] != 'Long') & (XMR['Direction'] != 'Short')][['Dec', 'Direction', 'Unnamed: 0']]
xm['coinname'] = 'XMR'

df = pd.concat([df, b, e, l, d, x, do, s, xm ])

In [263]:
df['Unnamed: 0'] = df['Unnamed: 0'].astype(int)

In [264]:
df = df.reset_index(drop=True)

In [265]:
labeledRow = []

for idx, row in df['Dec'].iteritems():
    labeledRow.append(TaggedDocument(utils.to_unicode(row).split(), ['Text' + '_%s' % str(idx)]))

In [266]:
docModel = Doc2Vec(min_count=0, window=5, vector_size=150, sample=1e-4, negative=5, workers=4, epochs=5, seed=1)

In [267]:
docModel.build_vocab(labeledRow)

In [268]:
docModel.train(labeledRow, total_examples=docModel.corpus_count, epochs=docModel.iter)

C:\Users\saroj\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


In [269]:
X = np.zeros((df.shape[0], 150))

In [270]:
for i in range(df.shape[0]):
    X[i] = docModel.docvecs['Text_'+str(i)]

In [271]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [272]:
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [273]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 150)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 128)            142848    
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 128)            131584    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 128)            16512     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 128)            0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 128)            512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 128)            0         
__________

In [274]:
predict = loaded_model.predict(X.reshape(-1, 1,150 ))

In [275]:
df['Direction'] = (predict > 0.5).astype(int)

In [276]:
df['Direction']=df['Direction'].replace(1, 'Long')

In [277]:
df['Direction']= df['Direction'].replace(0, 'Short')

In [278]:
for idx, row in df.iterrows():
    coins[row['coinname']].at[row['Unnamed: 0'], 'Direction'] = row['Direction']

In [288]:
coins['BTC'].to_csv('BTC.csv')

In [289]:
coins['LTC'].to_csv('LTC.csv')

In [290]:
coins['ETH'].to_csv('ETH.csv')

In [291]:
coins['DOGE'].to_csv('DOGE.csv')

In [292]:
coins['DASH'].to_csv('DASH.csv')

In [293]:
coins['XRP'].to_csv('XRP.csv')

In [294]:
coins['XMR'].to_csv('XMR.csv')

In [295]:
coins['STR'].to_csv('STR.csv')